<a href="https://colab.research.google.com/github/reue-lly/big-data-engineering/blob/main/HOA-1.1/Pornobe_Hands_on_Act_1_2_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pornobe, Reuel Christian, M.

---



#Procedure


In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AdvancedDFOps").getOrCreate()


Upload the online_retail_ii.csv from Kaggle into your Google Drive.

Mount Google Drive in Colab and read the CSV:


In [96]:
retail_df = spark.read.csv("/content/drive/MyDrive/big-data-engineering/online_retail_II.csv", header=True, inferSchema=True)
retail_df.printSchema()

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)



1. Merge data from two years of sales into a single DataFrame.
2. First, split the dataset by year:
3. Perform a full outer join on the "Customer ID" column:

In [97]:
retail_df.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|2009-12-01 07:45:00| 1.25|    13085.0|United Kingdom|
| 489434|    22064|PINK DOUGHNUT TRI...|      24|2009-12-01 07:45:00| 1.65|    13085.0|United Kingdom|
| 489434|    21871| SAVE THE PLANET MUG|      24|2009-12-01 07:45:00| 1.2

In [98]:
from pyspark.sql.functions import year, col

df_2009 = retail_df.filter(year(col("InvoiceDate")) == 2009)
df_2010 = retail_df.filter(year(col("InvoiceDate")) == 2010)

# Full outer join on Customer ID

joined_df = df_2009.join(df_2010, on=["Customer ID"], how="full_outer")
# Keep only one "Customer ID"



In [99]:
df_2009.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|2009-12-01 07:45:00| 1.25|    13085.0|United Kingdom|
| 489434|    22064|PINK DOUGHNUT TRI...|      24|2009-12-01 07:45:00| 1.65|    13085.0|United Kingdom|
| 489434|    21871| SAVE THE PLANET MUG|      24|2009-12-01 07:45:00| 1.2

In [100]:
df_2010.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
| 493410|  TEST001|This is a test pr...|       5|2010-01-04 09:24:00|  4.5|    12346.0|United Kingdom|
|C493411|    21539|RETRO SPOTS BUTTE...|      -1|2010-01-04 09:43:00| 4.25|    14590.0|United Kingdom|
| 493412|  TEST001|This is a test pr...|       5|2010-01-04 09:53:00|  4.5|    12346.0|United Kingdom|
| 493413|    21724|PANDA AND BUNNIES...|       1|2010-01-04 09:54:00| 0.85|       NULL|United Kingdom|
| 493413|    84578|ELEPHANT TOY WITH...|       1|2010-01-04 09:54:00| 3.75|       NULL|United Kingdom|
| 493413|    21723|ALPHABET HEARTS S...|       1|2010-01-04 09:54:00| 0.85|       NULL|United Kingdom|
| 493414|    21844|      RETRO SPOT MUG|      36|2010-01-04 10:28:00| 2.5

Union Operations: A union stacks DataFrames vertically but only works if their schemas match exactly in column names, order, and data types.

In [101]:
union_df = df_2009.union(df_2010)


Advanced Column Operations: Functions like withColumn, year, and struct allow creation of new calculated fields and nested structures without altering the original data.

In [102]:
from pyspark.sql.functions import month, struct

retail_df = retail_df.withColumn("TotalPrice", col("Quantity") * col("Price")) \
                     .withColumn("Year", year(col("InvoiceDate"))) \
                     .withColumn("Month", month(col("InvoiceDate"))) \
                     .withColumn("ProductInfo", struct(col("StockCode"), col("Description")))


Working with Arrays & Maps: Aggregating values into arrays or maps enables storing complex, multi-value attributes inside a single DataFrame column.



In [103]:
from pyspark.sql.functions import collect_list, create_map

products_per_customer = retail_df.groupBy("Customer ID") \
    .agg(collect_list("Description").alias("ProductsBought"))

maps_per_customer = retail_df.groupBy("Customer ID") \
    .agg(collect_list("StockCode").alias("Codes"),
         collect_list("Quantity").alias("Quantities"))


Aggregations and Analytics: Grouping and using aggregate functions such as sum or window functions enables advanced insights like top customers and time-based trends.

In [104]:
from pyspark.sql.functions import sum as _sum

top_customers = retail_df.groupBy("Customer ID") \
    .agg(_sum("TotalPrice").alias("TotalSpent")) \
    .orderBy(col("TotalSpent").desc()) \
    .limit(5)


In [105]:
retail_df.show()


+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+----+-----+--------------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|        TotalPrice|Year|Month|         ProductInfo|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+----+-----+--------------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|              83.4|2009|   12|{85048, 15CM CHRI...|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|2009|   12|{79323P, PINK CHE...|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|2009|   12|{79323W,  WHITE C...|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1| 

# **Supplementary Activity**

### **1. Load the dataset into a DataFrame (retail_df) and display its schema to confirm correct column names and types**

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AdvancedDFOps").getOrCreate()


In [108]:
retail_df = spark.read.csv("/content/drive/MyDrive/big-data-engineering/online_retail_II.csv", header=True, inferSchema=True)
retail_df.printSchema()

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)



### 2. **Data Cleaning**

In [109]:
from pyspark.sql.functions import col

In [110]:
# i created a function just for counting null values of a column
def countNull(column_name):
    null_count = retail_df.filter(col(column_name).isNull()).count()
    return null_count

In [111]:
print('Null Counts:')
print('Customer ID: ',countNull("Customer ID"))
print('Invoice: ', countNull("Invoice"))
print('Price', countNull("Price"))

Null Counts:
Customer ID:  243007
Invoice:  0
Price 0


#### a. Remove rows with null in "Customer ID", "Invoice", or "Price".

In [112]:
retail_df = retail_df.dropna(subset=["Customer ID", "Invoice", "Price"])

In [113]:
print('Null Counts:')
print('Customer ID: ',countNull("Customer ID"))
print('Invoice: ', countNull("Invoice"))
print('Price', countNull("Price"))

Null Counts:
Customer ID:  0
Invoice:  0
Price 0


In [114]:
retail_df.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|2009-12-01 07:45:00| 1.25|    13085.0|United Kingdom|
| 489434|    22064|PINK DOUGHNUT TRI...|      24|2009-12-01 07:45:00| 1.65|    13085.0|United Kingdom|
| 489434|    21871| SAVE THE PLANET MUG|      24|2009-12-01 07:45:00| 1.2

#### b. Filter out transactions where Quantity or Price is less than or equal to zero.


In [117]:
retail_df = retail_df.filter((retail_df.Quantity > 0) | (retail_df.Price > 0))

In [118]:
retail_df.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|2009-12-01 07:45:00| 1.25|    13085.0|United Kingdom|
| 489434|    22064|PINK DOUGHNUT TRI...|      24|2009-12-01 07:45:00| 1.65|    13085.0|United Kingdom|
| 489434|    21871| SAVE THE PLANET MUG|      24|2009-12-01 07:45:00| 1.2

### **3. Feature Engineering**


#### a. Create a new column "TotalPrice" = Quantity * Price.

In [119]:
retail_df = retail_df.withColumn("TotalPrice", col("Quantity") * col("Price"))
retail_df.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|        TotalPrice|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|              83.4|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|100.80000000000001|
| 489434|    21232|STRAWBERRY CERAMI...|      24|2009-12-01 07:45:00| 1.25|    13085.0|United Kingdom|              30.0|
| 489434|    22064|PINK 

#### b. Extract "Year" and "Month" from InvoiceDate.


In [120]:
retail_df = retail_df.withColumn("InvoiceYear", year(col("InvoiceDate"))) \
                                       .withColumn("InvoiceMonth", month(col("InvoiceDate")))
retail_df.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+-----------+------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|        TotalPrice|InvoiceYear|InvoiceMonth|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+-----------+------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|              83.4|       2009|          12|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|       2009|          12|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|       2009|          12|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|100.800000000000

### **4. Segmentation by Spend**

 4.1 Group by "Customer ID" and compute:

1.   Total amount spent (sum(TotalPrice)).






In [121]:
from pyspark.sql.functions import sum, avg, count, min, max

In [122]:
groupBy = retail_df.groupBy("Customer ID").sum("TotalPrice")
groupBy.show()

+-----------+--------------------+
|Customer ID|     sum(TotalPrice)|
+-----------+--------------------+
|    17884.0|   3028.889999999997|
|    14285.0|  3158.6400000000003|
|    16822.0|  144.83999999999997|
|    16596.0|   579.6300000000001|
|    17072.0|              282.05|
|    12671.0|   2622.481000000001|
|    16981.0|            -4620.86|
|    14452.0|              665.59|
|    12737.0|              3710.5|
|    15893.0|  305.28000000000003|
|    14094.0|              334.27|
|    14269.0|  261.68000000000006|
|    12467.0|-2.13162820728030...|
|    16916.0|              1123.4|
|    13607.0|  1060.6099999999997|
|    14024.0|              645.74|
|    13094.0|             2214.66|
|    17633.0|  1974.8899999999996|
|    15846.0|  107.01000000000002|
|    16656.0|  16307.720000000008|
+-----------+--------------------+
only showing top 20 rows



2. Number of unique purchases (countDistinct(Invoice)).





In [123]:
from pyspark.sql.functions import countDistinct

retail_df.select(countDistinct("Invoice")).show()

+-----------------------+
|count(DISTINCT Invoice)|
+-----------------------+
|                  44876|
+-----------------------+



#### 4.2 Create a new column "SpendingCategory":

##### a. "High" if total spent ≥ 5000.

In [125]:
retail_df= retail_df.withColumn("High", retail_df["TotalPrice"] >= 5000)
retail_df.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+-----------+------------+-----+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|        TotalPrice|InvoiceYear|InvoiceMonth| High|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+-----------+------------+-----+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|              83.4|       2009|          12|false|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|       2009|          12|false|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|       2009|          12|false|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    130

##### b. "Medium" if total spent between 1000 and 4999.

In [129]:
retail_df= retail_df.withColumn("Medium", (retail_df["TotalPrice"] <= 4999) & \
                                        (retail_df["TotalPrice"] >= 1000))
retail_df.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+-----------+------------+-----+------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|        TotalPrice|InvoiceYear|InvoiceMonth| High|Medium|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+-----------+------------+-----+------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|              83.4|       2009|          12|false| false|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|       2009|          12|false| false|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|       2009|          12|false| false|
| 489434|    22041|"RECORD FRAME 7""...|

#### c. "Low" if total spent < 1000.

In [130]:
retail_df= retail_df.withColumn("Low", retail_df["TotalPrice"] < 1000)
retail_df.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+-----------+------------+-----+------+----+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|        TotalPrice|InvoiceYear|InvoiceMonth| High|Medium| Low|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+-----------+------------+-----+------+----+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|              83.4|       2009|          12|false| false|true|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|       2009|          12|false| false|true|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|              81.0|       2009|          12|false| false|true|
| 489434| 

### **5. Top Products per Segment: For each "SpendingCategory", find the top 3 most purchased products (Description) based on total quantity sold.**